In [ ]:
import pandas as pd
from IPython.display import display, Markdown
import matplotlib.pyplot as plt

In [ ]:
caminho_arquivo = "../../dados/dados_siape_consolidado/receita_federal/cadastro_siape_receita_federal_mes_ano-2021-2023.csv"

df_rfb = pd.read_csv(caminho_arquivo)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
df_rfb['ano_mes'] = df_rfb['ano'].astype(str) + '-' + df_rfb['mes'].astype(str).str.zfill(2)

In [ ]:
df_rfb["REGIME_JURIDICO"].value_counts()

In [ ]:
df_rfb[(df_rfb["REGIME_JURIDICO"] == "REGIME JURIDICO UNICO") & (df_rfb["DESCRICAO_CARGO"] == "Inválido")]

In [ ]:
df_rfb[df_rfb["DESCRICAO_CARGO"] == "Inválido"].value_counts("REGIME_JURIDICO")

In [ ]:
df_consolidado = df_rfb.groupby(['UORG_LOTACAO', 'ano_mes', 'REGIME_JURIDICO']).size().reset_index(name='Quantidade')

In [ ]:
df_consolidado['REGIME_JURIDICO'].unique()

In [ ]:
df_consolidado.info()

In [ ]:
df_consolidado.value_counts("REGIME_JURIDICO")

In [ ]:
def dicionario_servidores_e_estagio_uorg_lotacao_ano_mes (df_rfb):

    dicionario_rfb = {}

    df_uorgs_com_estagio = df_rfb[df_rfb["REGIME_JURIDICO"].str.contains("estag", na=False, case=False)]

    uorgs_lotacao = df_uorgs_com_estagio["UORG_LOTACAO"].unique()

    contador = 0

    quantidade_uorg_lotacao = len(uorgs_lotacao)
    
    for uorg_lotacao in uorgs_lotacao:

        contador += 1

        contador_mes = 0

        df_uorg_lotacao = df_rfb[df_rfb["UORG_LOTACAO"] == uorg_lotacao]
        
        df_ano_mes_com_estagio = df_uorg_lotacao[df_uorg_lotacao["REGIME_JURIDICO"].str.contains("estag", na=False, case=False)]
        
        anos_meses = df_ano_mes_com_estagio["ano_mes"].unique()
        
        for ano_mes in anos_meses: 

            contador_mes += 1

            quantidade_ano_mes = len(anos_meses)

            display(f"Processando o uorg_lotacao: {uorg_lotacao} ({contador} de {quantidade_uorg_lotacao}) e mês: {ano_mes} ({contador_mes} de {quantidade_ano_mes}).")

            df_ano_mes = df_uorg_lotacao[df_uorg_lotacao['ano_mes'] == ano_mes]
            
            qtd_regime_unico = df_ano_mes[
                        df_ano_mes["REGIME_JURIDICO"].str.contains("regime juridico unico", na=False, case=False)]["Id_SERVIDOR_PORTAL"].count()
            
            qtd_estagiario =  df_ano_mes[
                        df_ano_mes["REGIME_JURIDICO"].str.contains("estag", na=False, case=False)]["Id_SERVIDOR_PORTAL"].count()
        
            if not uorg_lotacao in dicionario_rfb:
                dicionario_rfb[uorg_lotacao] = {}

            dicionario_rfb[uorg_lotacao][ano_mes] = {'regime_unico': qtd_regime_unico,'estagiarios': qtd_estagiario,'indice_estagio': qtd_estagiario/qtd_regime_unico}
    
    # Convertendo o dicionário em DataFrame
    df_resultado = pd.DataFrame.from_dict({(i, j): dicionario_rfb[i][j] 
                                           for i in dicionario_rfb.keys() 
                                           for j in dicionario_rfb[i].keys()},
                                          orient='index')

    df_resultado = df_resultado.reset_index().rename(columns={'level_0': 'uorg_lotacao', 'level_1': 'ano_mes'})
    
    return df_resultado

In [ ]:
df_totais = dicionario_servidores_e_estagio_uorg_lotacao_ano_mes(df_rfb)

In [ ]:
def gerar_grafico_indice_estagio (df, titulo):

    display (Markdown(f'# {titulo}'))
    
    # Criar gráfico de linha
    plt.figure(figsize=(10, 6))
    
    for org in df['uorg_lotacao'].unique():
        df_org = df[df['uorg_lotacao'] == org]
        x = df_org['ano_mes'].to_numpy()
        y = df_org['indice_estagio'].to_numpy()
        plt.plot(x, y, label=org)
    
    plt.title('Índice de Estágio ao longo do Tempo')
    plt.xlabel('Ano Mês')
    plt.ylabel('Índice de Estágio')
    plt.legend()
    plt.show()

In [ ]:
df_totais.to_csv('../../dados/dados_siape_consolidado/receita_federal/receita_federal_uorg_lotacao_com_estagio_2021_2023.csv', index=False)

In [ ]:
df_totais.sort_values("estagiarios", ascending=False)[0:25]

In [ ]:
df_contagem = df_totais.groupby("uorg_lotacao").mean("estagiario")

In [ ]:
df_totais.groupby("uorg_lotacao").size().sort_values(ascending=False)

In [ ]:
df_sem_informacao = df_rfb[df_rfb['UORG_LOTACAO'].str.contains('Sem informação', case=False, na=False)]

In [ ]:
display(Markdown('# Todas Uorgs rfb'))
display(df_rfb.value_counts('SITUACAO_VINCULO'))
display(Markdown('# Uorgs sem informação'))
display(df_sem_informacao.value_counts('SITUACAO_VINCULO'))

In [ ]:
display(df_contagem)

In [ ]:
gerar_grafico_indice_estagio (df_totais, "Índice estagiários/servidores nas unidades da RFB")

In [ ]:
df_estagio = df_rfb[df_rfb["REGIME_JURIDICO"].str.contains("estag", case=False, na=False)]

display(df_estagio)